In [1]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch import Tensor
import torch.nn.functional as F
from easydict import EasyDict as ED

from copy import deepcopy

In [2]:
from run_12ECG_classifier import run_12ECG_classifier, load_12ECG_model
from driver import load_challenge_data
from models.special_detectors import special_detectors
from models.ecg_crnn import ECG_CRNN
from model_configs.ecg_crnn import ECG_CRNN_CONFIG
from utils.misc import rdheader, ensure_lead_fmt
from utils.utils_signal import butter_bandpass_filter
from cfg import ModelCfg, TrainCfg
from data_reader import CINC2020Reader
from dataset import CINC2020
from utils.utils_nn import *

from eval_all import eval_all

%load_ext autoreload
%autoreload 2

In [ ]:
hehe_models = load_12ECG_model()

In [9]:
db_dir = "/media/cfs/wenhao71/data/cinc2020_data/"
working_dir = "./working_dir"

In [10]:
dr = CINC2020Reader(db_dir=db_dir,working_dir=working_dir)

log file path is set ./working_dir/working_dir/CINC2020-CINC2020.log
levels of c_handler and f_handler are set DEBUG


In [11]:
ds_config = deepcopy(TrainCfg)
ds_config.tranches_for_training

''

In [12]:
ds = CINC2020(config=ds_config,training=False)

log file path is set /media/cfs/wenhao71/workspace/cinc2020/working_dir/CINC2020-CINC2020.log
levels of c_handler and f_handler are set DEBUG


In [14]:
len(ds.records)

7343

In [ ]:
data_fp = dr.get_data_filepath(rec)
data_fp

In [ ]:
data, header_data = load_challenge_data(data_fp)

In [ ]:
run_12ECG_classifier(data, header_data, hehe_models, verbose=2)

In [ ]:
ann = dr.get_labels(rec, fmt='a')
ann

In [3]:
# eval_all(tranches="A")
eval_all()

classes (totally 12) for prediction:['AF', 'AFL', 'IAVB', 'IRBBB', 'LBBB', 'NSR', 'PAC', 'PVC', 'RBBB', 'SB', 'STach', 'TAb']
configuration of ECG_CRNN is as follows
{
    "cnn": {
        "name": "resnet_leadwise"
        "vgg16": {
            "num_convs": [
                2, 2, 3, 3, 3
            ]
            "num_filters": [
                144, 288, 576, 1152, 1152
            ]
            "groups": 1
            "kernel_initializer": "he_normal"
            "kw_initializer": {}
            "activation": "relu"
            "kw_activation": {}
            "block": {
                "filter_length": 3
                "subsample_length": 1
                "dilation": 1
                "batch_norm": True
                "pool_size": 3
                "pool_stride": 2
                "kernel_initializer": "he_normal"
                "kw_initializer": {}
                "activation": "relu"
                "kw_activation": {}
            }
        }
        "vgg16_mish": {
         

cnn output shape (batch_size, features, seq_len) = (None, 1536, 63)
clf_input_size = 1536
classes (totally 18) for prediction:['AF', 'AFL', 'IAVB', 'IRBBB', 'LAnFB', 'LBBB', 'LQT', 'NSIVCB', 'NSR', 'PAC', 'PVC', 'QAb', 'RBBB', 'SA', 'SB', 'STach', 'TAb', 'TInv']
configuration of ECG_CRNN is as follows
{
    "cnn": {
        "name": "resnet_leadwise"
        "vgg16": {
            "num_convs": [
                2, 2, 3, 3, 3
            ]
            "num_filters": [
                144, 288, 576, 1152, 1152
            ]
            "groups": 1
            "kernel_initializer": "he_normal"
            "kw_initializer": {}
            "activation": "relu"
            "kw_activation": {}
            "block": {
                "filter_length": 3
                "subsample_length": 1
                "dilation": 1
                "batch_norm": True
                "pool_size": 3
                "pool_stride": 2
                "kernel_initializer": "he_normal"
                "kw_initialize

cnn output shape (batch_size, features, seq_len) = (None, 1536, 63)
clf_input_size = 1536
log file path is set /media/cfs/wenhao71/workspace/cinc2020/working_dir/CINC2020-CINC2020.log
levels of c_handler and f_handler are set DEBUG
log file path is set /media/cfs/wenhao71/workspace/cinc2020/working_dir/CINC2020-CINC2020.log
levels of c_handler and f_handler are set DEBUG


100%|██████████| 7343/7343 [9:59:21<00:00,  4.61s/it]   


gathering results into a `DataFrame`...
- AUROC and AUPRC...
- Accuracy...
- F-measure...
- F-beta and G-beta measures...
- Challenge metric...
Done.

        results on tranches all:
        ------------------------------
        auroc:              0.8248985495216696
        auprc:              0.3346334308953883
        accuracy:           0.2014163148576876
        f_measure:          0.3432820832829895
        f_beta_measure:     0.3508813391065717
        g_beta_measure:     0.19770427468430604
        challenge_metric:   0.34332626918765097
        ----------------------------------------
    


,AF,AFL,IAVB,IRBBB,LAnFB,LBBB,LPR,LQT,NSIVCB,NSR,...,STach,TAb,TInv,Brady,LAD,RAD,PR,LQRSV,binary_predictions,truth_labels
0,0.791171,0.033081,0.021419,0.015358,0.008890,0.013908,0.005618,0.043320,0.013495,0.014519,...,0.101821,0.225539,0.011988,0.0,0.0,0.0,0.0,0.0,[AF],[AF]
1,0.539979,0.020361,0.026550,0.009917,0.007667,0.013193,0.004786,0.038812,0.059094,0.024954,...,0.809880,0.975959,0.109984,0.0,0.0,0.0,0.0,1.0,"[AF, STach, TAb, LQRSV]",[AF]
2,0.016977,0.011469,0.013483,0.019689,0.867941,0.014287,0.008135,0.010019,0.015333,0.960113,...,0.012309,0.038118,0.054898,0.0,0.0,0.0,0.0,0.0,"[LAnFB, NSR, RBBB]",[RBBB]
3,0.233598,0.012637,0.007963,0.467067,0.014960,0.018059,0.007950,0.007539,0.061538,0.046938,...,0.334069,0.314579,0.010693,0.0,0.0,0.0,0.0,1.0,"[RBBB, LQRSV]",[RBBB]
4,0.029407,0.014900,0.023862,0.018031,0.010142,0.005429,0.003650,0.032192,0.010765,0.735535,...,0.012835,0.012101,0.008859,0.0,0.0,0.0,0.0,0.0,"[NSR, RBBB]",[RBBB]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7338,0.038021,0.012790,0.026476,0.036903,0.012303,0.006092,0.005881,0.053683,0.023174,0.407567,...,0.011709,0.012343,0.007552,0.0,0.0,0.0,0.0,1.0,"[PAC, SB, LQRSV]",[SB]
7339,0.014340,0.008061,0.018057,0.010787,0.869412,0.013061,0.005232,0.005579,0.004509,0.779371,...,0.022919,0.034113,0.009564,0.0,0.0,0.0,0.0,1.0,"[LAnFB, NSR, SB, LQRSV]","[LAD, SB]"
7340,0.007477,0.008042,0.148558,0.017423,0.010595,0.075792,0.007289,0.007697,0.024087,0.935569,...,0.005223,0.022492,0.008504,0.0,0.0,0.0,0.0,1.0,"[NSR, LQRSV]",[NSR]
7341,0.012162,0.012000,0.134766,0.009586,0.005159,0.009400,0.007345,0.084620,0.007475,0.746070,...,0.019159,0.024067,0.027898,0.0,0.0,0.0,0.0,1.0,"[NSR, PAC, SB, LQRSV]","[SB, PAC, PAC]"


In [4]:
_

,AF,AFL,IAVB,IRBBB,LAnFB,LBBB,LPR,LQT,NSIVCB,NSR,...,STach,TAb,TInv,Brady,LAD,RAD,PR,LQRSV,binary_predictions,truth_labels
0,0.791171,0.033081,0.021419,0.015358,0.008890,0.013908,0.005618,0.043320,0.013495,0.014519,...,0.101821,0.225539,0.011988,0.0,0.0,0.0,0.0,0.0,[AF],[AF]
1,0.539979,0.020361,0.026550,0.009917,0.007667,0.013193,0.004786,0.038812,0.059094,0.024954,...,0.809880,0.975959,0.109984,0.0,0.0,0.0,0.0,1.0,"[AF, STach, TAb, LQRSV]",[AF]
2,0.016977,0.011469,0.013483,0.019689,0.867941,0.014287,0.008135,0.010019,0.015333,0.960113,...,0.012309,0.038118,0.054898,0.0,0.0,0.0,0.0,0.0,"[LAnFB, NSR, RBBB]",[RBBB]
3,0.233598,0.012637,0.007963,0.467067,0.014960,0.018059,0.007950,0.007539,0.061538,0.046938,...,0.334069,0.314579,0.010693,0.0,0.0,0.0,0.0,1.0,"[RBBB, LQRSV]",[RBBB]
4,0.029407,0.014900,0.023862,0.018031,0.010142,0.005429,0.003650,0.032192,0.010765,0.735535,...,0.012835,0.012101,0.008859,0.0,0.0,0.0,0.0,0.0,"[NSR, RBBB]",[RBBB]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7338,0.038021,0.012790,0.026476,0.036903,0.012303,0.006092,0.005881,0.053683,0.023174,0.407567,...,0.011709,0.012343,0.007552,0.0,0.0,0.0,0.0,1.0,"[PAC, SB, LQRSV]",[SB]
7339,0.014340,0.008061,0.018057,0.010787,0.869412,0.013061,0.005232,0.005579,0.004509,0.779371,...,0.022919,0.034113,0.009564,0.0,0.0,0.0,0.0,1.0,"[LAnFB, NSR, SB, LQRSV]","[LAD, SB]"
7340,0.007477,0.008042,0.148558,0.017423,0.010595,0.075792,0.007289,0.007697,0.024087,0.935569,...,0.005223,0.022492,0.008504,0.0,0.0,0.0,0.0,1.0,"[NSR, LQRSV]",[NSR]
7341,0.012162,0.012000,0.134766,0.009586,0.005159,0.009400,0.007345,0.084620,0.007475,0.746070,...,0.019159,0.024067,0.027898,0.0,0.0,0.0,0.0,1.0,"[NSR, PAC, SB, LQRSV]","[SB, PAC, PAC]"


In [5]:
df_eval_res = _

In [18]:
df_eval_res.to_csv("./log/full_eval_v0.2.csv")

In [8]:
df_eval_res.iloc[7339]

AF                                    0.01434
AFL                                0.00806124
IAVB                                0.0180573
IRBBB                               0.0107866
LAnFB                                0.869412
LBBB                                0.0130608
LPR                                0.00523215
LQT                                0.00557936
NSIVCB                             0.00450879
NSR                                  0.779371
PAC                                  0.187586
PVC                                 0.0220394
QAb                                0.00648522
RBBB                                0.0142287
SA                                  0.0082547
SB                                   0.948305
STach                               0.0229192
TAb                                 0.0341128
TInv                               0.00956383
Brady                                       0
LAD                                         0
RAD                               

In [16]:
df_eval_res.index = ds.records

In [17]:
df_eval_res

,AF,AFL,IAVB,IRBBB,LAnFB,LBBB,LPR,LQT,NSIVCB,NSR,...,STach,TAb,TInv,Brady,LAD,RAD,PR,LQRSV,binary_predictions,truth_labels
A0004,0.791171,0.033081,0.021419,0.015358,0.008890,0.013908,0.005618,0.043320,0.013495,0.014519,...,0.101821,0.225539,0.011988,0.0,0.0,0.0,0.0,0.0,[AF],[AF]
A0007,0.539979,0.020361,0.026550,0.009917,0.007667,0.013193,0.004786,0.038812,0.059094,0.024954,...,0.809880,0.975959,0.109984,0.0,0.0,0.0,0.0,1.0,"[AF, STach, TAb, LQRSV]",[AF]
A0010,0.016977,0.011469,0.013483,0.019689,0.867941,0.014287,0.008135,0.010019,0.015333,0.960113,...,0.012309,0.038118,0.054898,0.0,0.0,0.0,0.0,0.0,"[LAnFB, NSR, RBBB]",[RBBB]
A0027,0.233598,0.012637,0.007963,0.467067,0.014960,0.018059,0.007950,0.007539,0.061538,0.046938,...,0.334069,0.314579,0.010693,0.0,0.0,0.0,0.0,1.0,"[RBBB, LQRSV]",[RBBB]
A0035,0.029407,0.014900,0.023862,0.018031,0.010142,0.005429,0.003650,0.032192,0.010765,0.735535,...,0.012835,0.012101,0.008859,0.0,0.0,0.0,0.0,0.0,"[NSR, RBBB]",[RBBB]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
E10318,0.038021,0.012790,0.026476,0.036903,0.012303,0.006092,0.005881,0.053683,0.023174,0.407567,...,0.011709,0.012343,0.007552,0.0,0.0,0.0,0.0,1.0,"[PAC, SB, LQRSV]",[SB]
E10327,0.014340,0.008061,0.018057,0.010787,0.869412,0.013061,0.005232,0.005579,0.004509,0.779371,...,0.022919,0.034113,0.009564,0.0,0.0,0.0,0.0,1.0,"[LAnFB, NSR, SB, LQRSV]","[LAD, SB]"
E10332,0.007477,0.008042,0.148558,0.017423,0.010595,0.075792,0.007289,0.007697,0.024087,0.935569,...,0.005223,0.022492,0.008504,0.0,0.0,0.0,0.0,1.0,"[NSR, LQRSV]",[NSR]
E10334,0.012162,0.012000,0.134766,0.009586,0.005159,0.009400,0.007345,0.084620,0.007475,0.746070,...,0.019159,0.024067,0.027898,0.0,0.0,0.0,0.0,1.0,"[NSR, PAC, SB, LQRSV]","[SB, PAC, PAC]"
